In [1]:
from django.http.response import Http404, HttpResponseRedirect
from django.urls import reverse
from django.shortcuts import render, get_list_or_404, get_object_or_404
from django.views.decorators.clickjacking import xframe_options_sameorigin
from django.http import HttpResponse, QueryDict

import os, sys
os.chdir('/Users/wxy/OneDrive/projects/website/gathery')
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from user.models import User
from utility.models import City
from gallery.models import Gallery,Warehouse,Artist,Art,GU_Relation_old,GU_Relation,WU_Relation
from django.utils import timezone
from datetime import datetime,timedelta
import json,random,string
from itertools import islice
try:
    import zoneinfo
except ImportError:
    from backports import zoneinfo
from django.utils.dateparse import parse_datetime
from django.core.exceptions import ObjectDoesNotExist
from django.core.exceptions import ValidationError



In [2]:
a = Art.objects.filter(id__contains='aetG7NGd').first()
if a.picture == '':
    print('no file')
else:
    print(type(a.picture))

<class 'django.db.models.fields.files.ImageFieldFile'>


In [3]:
art.picture

NameError: name 'art' is not defined

In [4]:
a = '1'
type('2_{0}'.format(a))

str

In [5]:
from pprint import pprint
print(type(timezone.get_current_timezone()))
print(type(pytz.timezone(timezone.get_current_timezone_name())))
for key,value in timezone.get_current_timezone()._tzinfos.items():
    print(key)
    print(type(key))
# pprint(vars(timezone.get_current_timezone()))

<class 'backports.zoneinfo.ZoneInfo'>


NameError: name 'pytz' is not defined

In [6]:
from pprint import pprint
import pytz
g = GU_Relation.objects.get(staff_member__urlCode = 1032307067, gallery__galleryId = '9SDB2VDJ5P')
#pprint(vars(g))
g.date_join.astimezone(timezone.get_current_timezone())

ModuleNotFoundError: No module named 'pytz'

In [7]:
from pprint import pprint
g = GU_Relation.objects.get(staff_member__urlCode = 1033071098, gallery__galleryId = '9SDB2VDJ5P')
pprint(vars(g))


DoesNotExist: GU_Relation matching query does not exist.

In [ ]:
#000 [num,upper,lower]
def randomId(N, opt):
    head = random.choices( (opt&0b001 )*string.ascii_lowercase + (opt&0b010 )*string.ascii_uppercase + ( 0b100 )*string.digits, k=1)[0]
    body = ''.join(random.choices( (opt&0b001 )*string.ascii_lowercase + (opt&0b010 )*string.ascii_uppercase + string.digits, k=N-1))
    return head + body


def getArtId():
    a = randomId(8,0b011)
    while Art.objects.filter(id = a).exists():
        a = randomId(8,0b011)
    return str(a)


def getGalleryId():
    a = randomId(10,0b010)
    while Gallery.objects.filter(galleryId = a).exists():
        a = randomId(8,0b011)
    return str(a)


def getXCode(name,base,r):
    f = {
        'art': (Art),
        'artist': (Artist),
        'gallery': (Gallery),
        'warehouse': (Warehouse),
    }[name]
    #print(f)
    bot = pow(10, r - 4)
    head = pow(10, r - 3) 
    a = random.randint(bot, head-1) + base*head

    while f.objects.filter(urlCode = a).exists():
        a = random.randint(bot, head-1) + base*head
    return int(a)

def getArtistCode():
    return getXCode('artist',355,7)

def getArtCode():
    return getXCode('art',201,10)

def getGalleryCode():
    return getXCode('gallery',627,7)

def getWarehouseCode():
    return getXCode('warehouse',628,8)


#get random user
def pickUserId():
    min_u = 10
    max_u = 15
    u_n = random.randint(min_u, max_u)
    maxx = list(User.objects.all().values_list('id',flat=True))
    rr = random.sample(maxx, u_n)
    return User.objects.filter(id__in = rr)


def randomDates(n):# for an art
    now = timezone.now()
    now = now.replace(tzinfo=timezone.get_current_timezone())
    a = random.sample(range(1,20),n)
    a.sort(reverse=True)
    return [(now - timedelta(days=f)) for f in a]


def pickWarehouse(w_n):
    w_l = list(Warehouse.objects.all())
    rr = random.sample(w_l, w_n)
    return rr



In [ ]:
arts = [
    {'name': 'Frank', 'date made': 'Riley', 'warehouse': 'frank@email.com', 'gallery': '123-456-7890', 'author': 'New York', 'rack': 'a', 'status': 1},
    {'name': 'Steve', 'date made': 'Brannigan', 'warehouse': 'steve@example.fr', 'gallery': '123-456-1290', 'author': 'California', 'rack': 'a', 'status': 0},
    {'name': 'Marie', 'date made': 'Ambrose', 'warehouse': 'marie@yahoo.org', 'gallery': '123-976-7890', 'author': 'Texas', 'rack': 'a', 'status': 1}
]
wh = '62868707'
for a in arts:
    a['id'] = getArtId()
    a['wh'] = Warehouse.objects.get(urlCode = wh)
    a['date made'] = timezone.now()
    a['urlCode'] = getArtCode()
    # warning to have duplicate urlCode here,

batch_size = 100
try :
    objs = []
    for a in arts:
        p = {}
        p_h = [
            ['name'],
            ['urlCode'],
            ['id'],
            ['date_made','date made'],
            ['date_add','date add'],
            ['warehouse','wh'],
            ['last_transit', 'last transit'],
            ['rack'],
            ['type'],
            ['author'],
            ['status']
        ]
        for h in p_h:
            if a.get(h[-1]) is not None:
                p[h[0]] = a.get(h[-1])
        objs.append(Art(**p))
    objs = (o for o in objs)
    while True:
        batch = list(islice(objs, batch_size))
        if not batch:
            break
        print(objs)
        print(batch)
        print(len(batch))
        
        Art.objects.bulk_create(batch, batch_size)
except KeyError:
    raise KeyError

In [ ]:

def checkGU(gId, un):
    try:
        g = Gallery.objects.get(galleryId = gId)
        u = User.objects.get(username = un)
        return GU_Relation.objects.filter(gallery = g, staff_member = u).exists()
    except ObjectDoesNotExist:
        return False

checkGU('9SDB2VDJ5P','wxy')

In [ ]:
#中文
#chr(random.randint(0x4E00, 0x9FA5))

In [ ]:
print('user ',User.objects.count())
print('gallery ',Gallery.objects.count())
print('Warehouse ',Warehouse.objects.count())
print('GU_',GU_Relation.objects.count())
print('WU_ ',WU_Relation.objects.count())
print('Sum',GU_Relation_old.objects.count())
print('art ',Art.objects.count())

In [ ]:
#get some art
from django.db.models import F
list(Art.objects.all()[:5].values('name', 'id', 'author', 'date_made', 'date_add', gly_name = F('wa_collection__gallery__name'), wh_name = F('wa_collection__warehouse__name'), rack = F('wa_collection__rack'), wa_status = F('wa_collection__status') ))

In [ ]:
#generate User
def generateUser(n):
    for i in range(n):
        u = User(username = randomId(10,0b111), password = '4b95459bbab668e960cf9c2cfc24506e7078bd89510abae298726bfe1a17f8c9')
        u.save()


In [ ]:
#generate Gallery & Warehouses & GU/WU_relations
random_g_size = 15
min_w = 30
max_w = 50
for i in range(random_g_size):
    w_n = random.randint(min_w, max_w)
    current_g = Gallery(warehouse_num = w_n, urlCode = getGalleryCode())
    current_g.save()
    g_id = current_g.id
    users = pickUserId()
    users_r2 = [users[number] for number in random.sample(range(1, len(users)), int(len(users)/6))]
    users_r1 = [user for user in users if user not in users_r2]

    #generate Warehouses
    w_ids = []
    for j in range(w_n):
        aw = Warehouse(gallery = current_g, urlCode = getWarehouseCode())
        aw.save()
        w_ids.append(aw)
    
    print('u1', len(users_r1) , 'u2', len(users_r2))

    for au in users_r1:
        current_g.staff.add(au)
        t1 = GU_Relation_old(gallery = current_g, staff_member = au, role = str(random.randint(0,3)))
        t1.save()
        
    
    for au in users_r2:
        w_len = len(w_ids)
        w_sample = random.sample(range(w_len), random.randint(1,w_len))
        for wi in w_sample:
            
            guo = GU_Relation_old(gallery = current_g, staff_member = au, warehouse = w_ids[wi])
            guo.save()
            
            guo2 = WU_Relation(staff_member = au, warehouse = w_ids[wi])
            guo2.save()

    print('gallery ', i+1 , 'generated')

        


